# 句向量抽樣

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


## 1. 匯入資料

In [19]:
path_ = r'E:\專題\AllData\training\final_training' 

path_test = r'E:\專題\AllData\training\final_training\測試集再sampling'
path_ori = r'E:\專題\AllData\training\final_training\原始資料'
path_imb = r'E:\專題\AllData\training\final_training\十萬imbal'

path_bal = r'E:\專題\AllData\training\final_training\較bal'
path_pca = r'E:\專題\AllData\training\final_training\PCA'
path_tse = r'E:\專題\AllData\training\final_training\tomek_smote_enn'
#path_tsne = r'E:\專題\AllData\training\final_training\tsne' 

In [22]:
%%time
with open(path_pca+ r"/X_test_total_pc20.pkl", "rb") as f:
    train = pickle.load(f)
    
with open(path_ori+ r"/y_test.pkl", "rb") as f:
    trainlabel = pickle.load(f)
    

Wall time: 489 ms


In [14]:
with open(path_ori+ r"/loc_train.pkl", "rb") as f:
    trainloc = pickle.load(f)

In [23]:
print(np.shape(train))
print(np.shape(trainlabel))

(82884, 20)
(82884,)


In [21]:
%%time
#group = pd.read_excel(path_ + r"/icon.xlsx", sheet_name = 0)
#prop = pd.read_excel(path_ + r"/icon.xlsx", sheet_name = 1)

emo = pd.read_excel(path_ + r"/7_icon49_0429.xlsx")

Wall time: 28.9 ms


## 匯入「已做好的表格」檢查數量

In [23]:
emo.head()

,Group,歐易,貝爾,柯南,大福,貝卡,Peng,Total,Prop
0,2,12804,19242,16704,27468,1455,29991,107664,0.139997
1,1,12344,18589,13007,25806,1144,17022,87912,0.114313
2,4,8174,11728,16061,15821,1401,14920,68105,0.088558
3,5,8900,10703,12078,16951,1035,14433,64100,0.083350
4,3,7431,10458,10383,20824,1004,12031,62131,0.080790


## 2. 40 group 各 random 取 600 筆

#### 匯出每個 group 的位置各是那些、sampling 抽到的位置
也可抽 39 個 group，但就是第 40 個 group 不會被抽到

In [24]:
def sampling(train, trainlabel, group, sampling):
    
    alldata = list(range(np.shape(train)[0]))  #資料總共幾筆
   
    # 表情 group 對應的位置
    allemo = []
    for j in range(1, group + 1):
        j_emo = [i for i in alldata if trainlabel[i]== j]
        allemo.append(j_emo)
        # allemo:[[group 1 表情符號位置], [group 2 表情符號位置], [], ...[group 40 表情符號位置]]
    
    
    #抽樣
    random_want = []
    for j in range(group):
        gg = np.random.choice(allemo[j], sampling, replace = False).tolist() #每個組隨機抽到的表情位置
        random_want = random_want + gg
        # random_want: [group 1 抽樣的位置, group 2 抽樣的位置 ...]
        # 匯出按照 group排序，label可直接上為 111..11222..22333...33 .....
        
    return allemo, random_want

In [25]:
print([trainlabel.count(i) for i in list(range(1, 20+ 1))])

[11096, 9644, 7185, 6533, 5868, 4416, 4582, 3340, 3252, 2912, 2531, 2587, 2301, 1572, 1817, 1193, 1112, 1179, 1065, 988]


<hr>

In [10]:
%%time
# 3人取前40類900筆
result = sampling(train, trainlabel, 20, 5000)

random_sample = result[1]   #抽樣的位置
data = train[random_sample, ]
data = np.array(data)
print(data.shape)

label = np.repeat(list(range(1, 20+1)), 5000)
print(len(label))

(100000, 20)
100000
Wall time: 747 ms


### 匯出抽樣資料

In [13]:
%%time
with open(path_bal + '/X_train_bal10.pkl'','wb') as f:
    pickle.dump(data, f)

with open(path_bal + '/y_train_bal10.pkl','wb') as f:
    pickle.dump(label, f)
    
#rand_emo = pd.DataFrame(data, index=False)
#rand_emo.to_excel(path_go + r'\all_emo600.xlsx')

Wall time: 3.24 s


In [17]:
with open(path_ + '/loc_train_bal10.pkl','wb') as f:
    pickle.dump(trainid, f)